In [ ]:
import requests
import numpy as np
import os, sys
import random
import math
import bisect
import arrow
import pytz
from datetime import datetime, date, timedelta
from dateutil import tz
from timezonefinder import TimezoneFinder
from matplotlib import pyplot as plt, dates
from matplotlib.ticker import *
from matplotlib_helper import *
from carbon_api_client import *
from typing import List

In [ ]:
enable_savefig = False

In [ ]:
def get_all_fuel_types(energy_mixture_timeseries):
    fuel_types = set()
    for entry in energy_mixture_timeseries:
        fuel_types.update(entry['power_by_fuel_type'].keys())
    return fuel_types

In [ ]:
def get_timeseries_by_fuel_type(energy_mixture_timeseries, fuel_types):
    d_timeseries_by_fuel_type = {}
    for fuel_type in fuel_types:
        d_timeseries_by_fuel_type[fuel_type] = []
    for entry in energy_mixture_timeseries:
        for fuel_type in fuel_types:
            value = entry['power_by_fuel_type'][fuel_type] if fuel_type in entry['power_by_fuel_type'] else 0.
            d_timeseries_by_fuel_type[fuel_type].append(value)
    return d_timeseries_by_fuel_type

In [ ]:
# Similar to the bar plot in supply trend in http://www.caiso.com/todaysoutlook/pages/supply.html

region = 'US-CAISO'
CA_GPS = (36.78, -119.78)

target_date = arrow.get(date(2022, 7, 3), tzinfo=tz.gettz('America/Los_Angeles'))
energy_mixture_timeseries = call_sysnet_energy_mixture_api(CA_GPS[0], CA_GPS[1], target_date, target_date.shift(days=1))
# print(energy_mixture_timeseries[:2])

timestamps = [e['timestamp'] for e in energy_mixture_timeseries]
# fuel_types = get_all_fuel_types(energy_mixture_timeseries)
fuel_types = { fuel_type for entry in energy_mixture_timeseries for fuel_type in entry['power_by_fuel_type'] }
d_fuel_type_timeseries = get_timeseries_by_fuel_type(energy_mixture_timeseries, fuel_types)

print('fuel_types', fuel_types)
# print('timestamps len:', len(timestamps))
# for fuel_type in d_fuel_type_timeseries:
#     print(fuel_type, 'len:', len(d_fuel_type_timeseries[fuel_type]))

plt.figure(figsize=(8, 4.8))
fuel_order = []
plot_colors = {
    'coal': 'black',
    'gas': '#F2915F',
    'nuclear': '#808080',
    'geothermal': 'brown',
    'biomass': 'yellowgreen',
    'hydro': '#40CDC4',
    'wind': 'royalblue',
    'solar': 'gold',
}
plot_stacked_line(timestamps, d_fuel_type_timeseries, order=plot_colors.keys(), d_label_colors=plot_colors)

plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.title(f'Energy in MW by fuel type in {region} on {target_date.strftime("%Y-%m-%d")}')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.legend(loc='center left', bbox_to_anchor=(1.04, 0.5), ncol=1)
plt.xticks(rotation=15)
# plt.ylim(0, 1000)
plt.tight_layout()
savefig_filename = 'energy-by-fuel-mix.%s.%s.png' % (target_date.strftime("%Y-%m-%d"), region)
if enable_savefig:
    plt.savefig(savefig_filename)

In [ ]:
def get_total_amount(write_ratio, run_count):
    return 1 + write_ratio * (run_count - 1)

In [ ]:
# I/O data size impact

plot_application_datapoints = True

# l_wan_bandwidth = [
#     (125/1024, '125 Mbps (average TCP)'),
#     (5, '5 Gbps (max VM)'),
# ]

# l_wan_bandwidth = [
#     (20/1024, '20 Mbps (min TCP)'),
#     (400/1024, '400 Mbps (max TCP)'),
#     (5, '5 Gbps (max VM)'),
# ]

l_wan_bandwidth = [
    (125/1024, 'Single run (125 Mbps)'),
    (125/1024 / (get_total_amount(0.05, 7) / 7), '7x run, 5% new data'),
    (125/1024 / (get_total_amount(0.5, 7) / 7), '7x run, 50% new data'),
    (125/1024 / (get_total_amount(0.05, 30) / 30), '30x run, 5% new data'),
    (125/1024 / (get_total_amount(0.5, 30) / 30), '30x run, 50% new data'),
]

# On our Xeon gold 6138 CPUs
cpu_hours_per_kj = 1/22.5
m_app_kj_per_gb = {
    'Compression (gzip)': 0.47,
    # 'Compression (bzip)': 0.83,
    'Compile Linux': 76.42,
    'Video resizing (4k -> 1080p, h.264)': 1.41,
    'Video effect (4k, grayscale, h.264)': 11.53,
    'Video effect (4k, grayscale, h.265)': 96.8,
    # 'Video transcoding (h.264 -> h.265)': 19.54,
    # 'Video transcoding (h.265 -> h.264)': 82.67,
    'Single model ML inference': 1800,
}

YMAX = 1.25
BITS_PER_BYTE = 8
TRANSFER_POWER_PERCENTAGE = 0.30

def normalize_migration_overhead(compute_cpu_hours: float, data_in_gb: float, bandwidth_gbps):
    transfer_time_hours = data_in_gb * BITS_PER_BYTE / bandwidth_gbps / timedelta(hours=1).total_seconds()
    transfer_cpu_hours = transfer_time_hours * TRANSFER_POWER_PERCENTAGE
    return transfer_cpu_hours / compute_cpu_hours

plt.figure(figsize=(8, 4.8))
x = np.logspace(-4, 2)
ones = np.ones(len(x))
for wan_bandwidth_gbps, label in l_wan_bandwidth:
    y = list(map(normalize_migration_overhead, x, ones, ones * wan_bandwidth_gbps))
    plt.plot(x, y, label=label)
plt.xscale('log')
plt.hlines(1, x[0], x[-1], color='black', linestyles='dotted')
plt.yscale('log')
plt.ylim(None, YMAX)
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, pos: '{:.3g}%'.format(x * 100)))
plt.xlabel('CPU hours / GB')
plt.ylabel('Overhead (%)')
ax.grid(axis='y', which='major', alpha=0.5)
ax.grid(axis='y', which='minor', alpha=0.2)

if plot_application_datapoints:
    for app, kj_per_gb in m_app_kj_per_gb.items():
        cpu_hours_per_gb = kj_per_gb * cpu_hours_per_kj
        plt.vlines(cpu_hours_per_gb, 0, YMAX, color='gray', linestyles='dashed')
        plt.text(cpu_hours_per_gb, YMAX, app, rotation=30)

plt.legend(loc='lower left')


In [ ]:
l_write_ratio = [
    0,
    0.05,
    0.5,
    1,
]

l_run_count = np.array(range(30)) + 1

for write_ratio in l_write_ratio:
    x = l_run_count
    y = list(map(lambda run_count: get_total_amount(write_ratio, run_count) / run_count, l_run_count))
    plt.plot(x, y, label=f'{write_ratio * 100:.0f}% new data', marker='.')
plt.xlabel('# of runs')
plt.ylabel('Amortized data transfer per run')
plt.ylim(0, 1.05)
ax = plt.gca()
ax.grid(axis='y', which='major', alpha=0.5)
ax.grid(axis='y', which='minor', alpha=0.2)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, pos: '{:.3g}%'.format(x * 100)))
plt.legend()